In [1]:
from DataLoader import MyOwnDataloader
from pycocotools.coco import COCO

import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import torchvision

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
dataDir='/media/gamedisk/COCO_dataset/'
val='val2017'
train = 'train2017'

val_annFile='{}/annotations/instances_{}.json'.format(dataDir,val)
train_annFile='{}/annotations/instances_{}.json'.format(dataDir,train) 
# Batch size
batch_size = 8



classes = {
    "bird": 0,
    "cat": 1,
    "dog": 2,
    "horse": 3,
    "sheep": 4,
    "cow": 5,
    "elephant": 6,
    "bear": 7,
    "zebra": 8,
    "giraffe": 9
}


coco = COCO(val_annFile)
val_loader = MyOwnDataloader(dataDir = dataDir, dataType = val,
                     annFile = val_annFile, classes = classes, train_batch_size=batch_size)
data_loader = val_loader.concat_datasets()


loading annotations into memory...
Done (t=0.38s)
creating index...
index created!
loading annotations into memory...
Done (t=0.33s)
creating index...
index created!
bird 0 [16] 55299


In [4]:
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

def get_model_instance_segmentation(num_classes):
    # load an instance segmentation model pre-trained pre-trained on COCO
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=False)
    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    return model
    

# 2 classes; Only target class or background

num_epochs = 2
model = get_model_instance_segmentation(10)

# move model to the right device
model.to(device)
    
# parameters
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)

len_dataloader = len(data_loader)

for epoch in range(num_epochs):
    model.train()
    i = 0 
    print(f'epoch # {epoch}')   
    for i, (imgs, annotations) in tqdm(enumerate(data_loader)):
        i += 1
        imgs = list(img.to(device) for img in imgs)
        annotations = [{k: v.to(device) for k, v in t.items()} for t in annotations]
        loss_dict = model(imgs, annotations)
        losses = sum(loss for loss in loss_dict.values())

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        if i%5:
            print(f'Iteration: {i}/{len_dataloader}, Loss: {losses}')

epoch # 0


1it [00:00,  1.07it/s]

Iteration: 1/137, Loss: 10.653388977050781


2it [00:01,  1.55it/s]

Iteration: 2/137, Loss: 13.098834991455078


...

56it [00:21,  2.61it/s]

Iteration: 56/137, Loss: nan


57it [00:22,  2.54it/s]

Iteration: 57/137, Loss: nan


KeyboardInterrupt: 